## Summary: Building Long-Term Memory with LangMem  
Overview  
This demo explains how to combine vector database storage with an agent framework to create long-term memory using LangMem, a library built by the LangChain team. The goal is to enable agents to remember facts across multiple interactions.

Key Steps Covered  
1. Setting up LangMem  
LangMem handles the memory store behind the scenes, using vector embeddings to save and retrieve information.

Memory storage is backed by OpenAI embeddings in this example.


from langmem import OpenAIMemoryStore

store = OpenAIMemoryStore()

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain_core.messages import HumanMessage

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
)

## 2. Creating the Workflow  
A ReAct agent is created using a prebuilt LangMem utility:

create_react_agent(model, tools, store)
No custom StateSchema is needed.

Two tools are defined:  

manage_memory_tool: Saves information into the store.

search_memory_tool: Retrieves information from the store.


tools = [manage_memory_tool, search_memory_tool]

agent = create_react_agent(model=llm, tools=tools, store=store)
The agent workflow:

Agent node → Tools node → Agent node → End

In [ ]:
agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[
        # Memory tools use LangGraph's BaseStore for persistence
        create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),
    ],
    store=store,
)

In [ ]:
agent

## 3. Inspecting the Input Schema  
The input schema for the agent expects a field called messages.

messages should include:  

SystemMessage

HumanMessage

AIMessage

ToolMessages (after tool calls)


{

  "messages": [...]

}

In [ ]:
agent.get_input_jsonschema()

## 4. Example Interaction: Setting and Retrieving Preferences  
a. Initial Question
User asks:  

"What are my lighting preferences?"  
The agent:  

Calls the search_memory tool.

Tool responds: No memory found.

Agent replies: "I don't have any information about your lighting preferences."
  
b. Saving New Information
User follows up:
  
"Remember that I prefer dark mode."  
The agent:

Calls the manage_memory tool.

Saves the new preference into the store.

Replies: "I've noted that you prefer dark mode."
  
c. New Session Retrieval
A new thread is started with a fresh message:

"What are my lighting preferences?"
Only this question is provided (no prior conversation).

The agent:

Calls the search_memory tool.

Finds the saved memory from the previous interaction.
  
Replies correctly: "You prefer dark mode."

In [ ]:
# Ask a random question to the agent
output = agent.invoke(
    input = {
        "messages": [
            {
                "role": "user", 
                "content": "What are my lighting preferences?"
            }
        ]
    },
    config = {"configurable": {"thread_id": "1"}}
)

output['messages']

In [ ]:
messages = output['messages']

In [ ]:
messages.append(HumanMessage("Ok. Remember that I prefer dark mode."))

In [ ]:
# Store a new memory in one Session
output = agent.invoke(
    input = {
        "messages": messages,
    },
    config = {"configurable": {"thread_id": "1"}}
)

output['messages']

In [ ]:
# Retrieve the stored memory in another Session
output = agent.invoke(
    {
        "messages": [
            {
                "role": "user", 
                "content": "What are my lighting preferences?"
            }
        ]
    },
    config = {"configurable": {"thread_id": "2"}}
)

output['messages']

## 5. Key Concepts Highlighted  
Persistent memory across different sessions and thread IDs.

Memory search and management handled with tools abstracted from the agent's reasoning.

Vector-based retrieval ensures memory is stored in a scalable, semantically searchable format.

The user does not need to manually maintain history; it's offloaded to LangMem.
  
## 6. Conclusion  
LangMem makes it easy to extend ReAct agents with long-term memory.

Agents can now recall user preferences or facts across completely different conversations.

This memory-enhanced design is a key step toward building more lifelike, persistent AI agents.